# BiH Political Data Collection

Voting data for the following years obtained from the [Central Election Commission BiH](https://www.izbori.ba/?Lang=6). Used to control for ethnic concentration and policy preferences.
* [2010](https://www.izbori.ba/Finalni2010/Finalni/PredsjednistvoBiH/Nivo701702.aspx)
* [2014](https://www.izbori.ba/Potvrdjeni2014/Finalni/PredsjednistvoBiH/Nivo703.aspx)
* [2018](https://www.izbori.ba/rezultati_izbora?resId=25&langId=4#/1/2/0/0/703)
* [2022](https://www.izbori.ba/Rezultati_izbora/?resId=32&langId=4#/1/1/0/0/701)

I will be focusing on the voting data associated with the presidential elections of the country. For reference, there are three presidents in BiH, one for each major ethnic group and the Bosniak and Croat ones are voted for in FBiH. As my analysis is limited to FBiH, I will only collect data on those two presidential races.

The great thing about the presidential voting data is that the website classifies each candidate to be a Bosniak or Croat so I can avoid needing to make the decision of which party or candidate represents a specific ethnic group.

I will be scraping (or calling APIs for) the data from the above websites and as they do not have a robots.txt, I will assume they are okay with me doing this.

## Prerequisites

In [1]:
import json
import requests
import sqlite3

import bs4 as bs
import pandas as pd

from tools.scrape import (
    create_db, 
    insert_candidates, 
    insert_year,
    get_year_id,
    insert_municipalities,
    get_municipality_id_per_year,
    get_candidate_ids,
    insert_candidate_results,
    modern_scrape,
    older_scrape,
)
from tools.config import SAVE_DIR

Going to set up the schema for the database and save the db file in the processed folder. Details on the implemenation of most helper functions can be found in `notebooks/tools/scrape.py`.

In [2]:
create_db()

## 2022

I will be starting with this since it is very similar to the 2018 website and is more modern than the 2014 or 2010 website which will likely require a different scraping script.

### 1: Insert into Years

To make inserting into the db faster I will prefill it with municipality and year.

In [3]:
insert_year(2022)

Successfully inserted 1 year.


### 2: Insert into Municipalities

I got the following html from inspecting the dropdown for municipalities. The codes are what is important here.

In [4]:
scroll_down_html = """
<select class="ddlMenu ng-pristine ng-valid" ng-model="electoralUnitSelectedId" ng-change="race_1_ElectoralUnitChange()">
                        <option value="0">-</option>
                        <!-- ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="1" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">VELIKA KLADUŠA (001)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="true" ng-repeat="eu in electoralUnitList" value="2" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope" selected="selected">CAZIN (002)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="3" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">BIHAĆ (003)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="4" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">BOSANSKA KRUPA (004)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="5" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">BUŽIM (005)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="17" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">ODŽAK (017)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="20" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">DOMALJEVAC - ŠAMAC (020)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="22" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">ORAŠJE (022)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="25" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">GRADAČAC (025)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="27" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">BRČKO DISTRIKT BIH (OPCIJA FBIH) (027)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="30" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">BOSANSKI PETROVAC (030)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="32" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">SANSKI MOST (032)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="36" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">DOBOJ ISTOK (036)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="37" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">DOBOJ JUG (037)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="39" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">TEŠANJ (039)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="42" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">MAGLAJ (042)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="44" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">GRAČANICA (044)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="47" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">LUKAVAC (047)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="49" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">SREBRENIK (049)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="50" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">TUZLA (050)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="52" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">ČELIĆ (052)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="55" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">TEOČAK (055)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="57" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">DRVAR (057)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="59" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">KLJUČ (059)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="65" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">JAJCE (065)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="67" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">DOBRETIĆI (067)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="75" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">ŽEPČE (075)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="77" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">ZAVIDOVIĆI (077)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="78" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">BANOVIĆI (078)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="79" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">ŽIVINICE (079)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="80" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">KALESIJA (080)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="82" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">SAPNA (082)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="84" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">BOSANSKO GRAHOVO (084)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="85" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">GLAMOČ (085)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="89" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">DONJI VAKUF (089)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="91" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">TRAVNIK (091)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="93" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">ZENICA (093)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="94" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">KAKANJ (094)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="95" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">VAREŠ (095)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="96" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">OLOVO (096)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="98" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">KLADANJ (098)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="106" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">LIVNO (106)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="107" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">KUPRES (107)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="109" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">BUGOJNO (109)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="110" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">GORNJI VAKUF - USKOPLJE (110)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="111" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">NOVI TRAVNIK (111)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="112" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">VITEZ (112)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="113" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">BUSOVAČA (113)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="114" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">FOJNICA (114)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="115" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">KISELJAK (115)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="116" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">VISOKO (116)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="117" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">BREZA (117)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="118" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">ILIJAŠ (118)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="124" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">TOMISLAVGRAD (124)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="125" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">PROZOR - RAMA (125)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="126" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">JABLANICA (126)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="127" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">KONJIC (127)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="129" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">KREŠEVO (129)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="130" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">HADŽIĆI (130)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="131" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">ILIDŽA (131)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="133" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">NOVI GRAD SARAJEVO (133)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="135" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">VOGOŠĆA (135)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="136" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">CENTAR SARAJEVO (136)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="137" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">STARI GRAD SARAJEVO (137)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="139" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">NOVO SARAJEVO (139)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="141" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">TRNOVO (FBIH) (141)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="143" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">PALE (FBIH) (143)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="148" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">POSUŠJE (148)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="149" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">GRUDE (149)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="150" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">ŠIROKI BRIJEG (150)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="165" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">FOČA (FBIH) (165)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="167" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">GORAŽDE (167)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="171" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">LJUBUŠKI (171)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="172" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">ČITLUK (172)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="173" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">ČAPLJINA (173)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="174" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">NEUM (174)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="176" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">STOLAC (176)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="181" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">RAVNO (181)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="183" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">USORA (183)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="199" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">GRAD MOSTAR (199)</option><!-- end ngRepeat: eu in electoralUnitList -->
                    </select>
"""

In [5]:
soup = bs.BeautifulSoup(scroll_down_html)
municipality_to_code = {
    option.text.split("(")[0].strip().lower() : int(option.get("value"))
    for option in soup.find_all("option")
    if int(option.get("value")) != 0
}

I had some trouble with the websites having different ways of naming the municipalites on each different election so I had an LLM make this for me to have a standardized way of names to codes. 

In [6]:
utf8_ascii_municips = {
    "banovići": "banovici",
    "bihać": "bihac",
    "bosanska krupa": "bosanska krupa",
    "bosanski petrovac": "bosanski petrovac",
    "bosansko grahovo": "bosansko grahovo",
    "breza": "breza",
    "brčko distrikt bih": "brcko distrikt bih",
    "bugojno": "bugojno",
    "busovača": "busovaca",
    "bužim": "buzim",
    "cazin": "cazin",
    "centar sarajevo": "centar sarajevo",
    "doboj istok": "doboj istok",
    "doboj jug": "doboj jug",
    "dobretići": "dobretici",
    "domaljevac - šamac": "domaljevac samac",
    "donji vakuf": "donji vakuf",
    "drvar": "drvar",
    "fojnica": "fojnica",
    "foča": "foca",
    "glamoč": "glamoc",
    "goražde": "gorazde",
    "gornji vakuf - uskoplje": "gornji vakuf uskoplje",
    "grad mostar": "grad mostar",
    "gradačac": "gradacac",
    "gračanica": "gracanica",
    "grude": "grude",
    "hadžići": "hadzici",
    "ilidža": "ilidza",
    "ilijaš": "ilijas",
    "jablanica": "jablanica",
    "jajce": "jajce",
    "kakanj": "kakanj",
    "kalesija": "kalesija",
    "kiseljak": "kiseljak",
    "kladanj": "kladanj",
    "ključ": "kljuc",
    "konjic": "konjic",
    "kreševo": "kresevo",
    "kupres": "kupres",
    "livno": "livno",
    "ljubuški": "ljubuski",
    "lukavac": "lukavac",
    "maglaj": "maglaj",
    "neum": "neum",
    "novi grad sarajevo": "novi grad sarajevo",
    "novi travnik": "novi travnik",
    "novo sarajevo": "novo sarajevo",
    "odžak": "odzak",
    "olovo": "olovo",
    "orašje": "orasje",
    "pale": "pale",
    "posušje": "posusje",
    "prozor - rama": "prozor rama",
    "ravno": "ravno",
    "sanski most": "sanski most",
    "sapna": "sapna",
    "srebrenik": "srebrenik",
    "stari grad sarajevo": "stari grad sarajevo",
    "stolac": "stolac",
    "teočak": "teocak",
    "tešanj": "tesanj",
    "tomislavgrad": "tomislavgrad",
    "travnik": "travnik",
    "trnovo": "trnovo",
    "tuzla": "tuzla",
    "usora": "usora",
    "vareš": "vares",
    "velika kladuša": "velika kladusa",
    "visoko": "visoko",
    "vitez": "vitez",
    "vogošća": "vogosca",
    "zavidovići": "zavidovici",
    "zenica": "zenica",
    "čapljina": "capljina",
    "čelić": "celic",
    "čitluk": "citluk",
    "široki brijeg": "siroki brijeg",
    "žepče": "zepce",
    "živinice": "zivinice"
}

To go from old municipality names to standardized ones.

In [7]:
m_to_c = { 
    utf8_ascii_municips[municipality] : municipality_to_code[municipality] 
    for municipality in municipality_to_code.keys()
}

Now to insert the municipalites to the year.

In [8]:
municips = list(m_to_c)
insert_municipalities(2022, municips)

Successfully inserted 80 municipalites.


### 3: Insert into Candidates

Since this data is all on one page and to avoid making my scraping script more complicated I will do this by hand.

In [9]:
candidates = [
    ("DENIS BEĆIROVIĆ - UJEDINJENI ZA SLOBODNU BOSNU I HERCEGOVINU", "B"),
    ("BAKIR IZETBEGOVIĆ - SDA - STRANKA DEMOKRATSKE AKCIJE", "B"),
    ("MIRSAD HADŽIKADIĆ - PLATFORMA ZA PROGRES", "B"),
    ("ŽELJKO KOMŠIĆ - DEMOKRATSKA FRONTA - DF", "C"),
    ("BORJANA KRIŠTO - HDZ BIH-HRVATSKA DEMOKRATSKA ZAJEDNICA BOSNE I HERCEGOVINE", "C"),
]
insert_candidates(candidates)

Successfully inserted 5 candidates.


### 4: Insert election results into CandidateResults

In [10]:
year_key = "WebResult_2022GENT1_2022_4_20_14_10_43"
modern_scrape(2022, year_key, m_to_c)

Successfully inserted 5 candidate results for velika kladusa.
Successfully inserted 5 candidate results for cazin.
Successfully inserted 5 candidate results for bihac.
Successfully inserted 5 candidate results for bosanska krupa.
Successfully inserted 5 candidate results for buzim.
Successfully inserted 5 candidate results for odzak.
Successfully inserted 5 candidate results for domaljevac samac.
Successfully inserted 5 candidate results for orasje.
Successfully inserted 5 candidate results for gradacac.
Successfully inserted 5 candidate results for brcko distrikt bih.
Successfully inserted 5 candidate results for bosanski petrovac.
Successfully inserted 5 candidate results for sanski most.
Successfully inserted 5 candidate results for doboj istok.
Successfully inserted 5 candidate results for doboj jug.
Successfully inserted 5 candidate results for tesanj.
Successfully inserted 5 candidate results for maglaj.
Successfully inserted 5 candidate results for gracanica.
Successfully insert

### 5: Sanity Check

In [11]:
try:
    conn = sqlite3.connect(SAVE_DIR + "political_data.db")

    sql_query = """
        SELECT
            m.name AS municipality_name,
            y.year,
            c.name,
            c.ethnicity,
            cr.votes
        FROM 
            Municipalities as m
        JOIN
            Years AS y ON m.year_id = y.year_id
        JOIN
            CandidateResults AS cr ON m.municipality_id = cr.municipality_id
        JOIN
            Candidates AS c ON cr.candidate_id = c.candidate_id
        WHERE
            y.year = 2022
    """

    df = pd.read_sql_query(sql_query, conn)
except sqlite3.Error as e:
    print(f"An error occurred: {e}")
finally:
    conn.close()

In [12]:
df

,municipality_name,year,name,ethnicity,votes
0,velika kladusa,2022,DENIS BEĆIROVIĆ - UJEDINJENI ZA SLOBODNU BOSNU...,B,4662
1,velika kladusa,2022,BAKIR IZETBEGOVIĆ - SDA - STRANKA DEMOKRATSKE ...,B,3039
2,velika kladusa,2022,ŽELJKO KOMŠIĆ - DEMOKRATSKA FRONTA - DF,C,2999
3,velika kladusa,2022,MIRSAD HADŽIKADIĆ - PLATFORMA ZA PROGRES,B,690
4,velika kladusa,2022,BORJANA KRIŠTO - HDZ BIH-HRVATSKA DEMOKRATSKA ...,C,190
...,...,...,...,...,...
395,grad mostar,2022,BORJANA KRIŠTO - HDZ BIH-HRVATSKA DEMOKRATSKA ...,C,21651
396,grad mostar,2022,ŽELJKO KOMŠIĆ - DEMOKRATSKA FRONTA - DF,C,9332
397,grad mostar,2022,DENIS BEĆIROVIĆ - UJEDINJENI ZA SLOBODNU BOSNU...,B,8822
398,grad mostar,2022,BAKIR IZETBEGOVIĆ - SDA - STRANKA DEMOKRATSKE ...,B,6157


The above checks out!

## 2018

### 1: Insert into Years

In [13]:
insert_year(2018)

Successfully inserted 1 year.


### 2: Insert into Municipalities

`municips` defined in 2022 above in the same step.

In [14]:
insert_municipalities(2018, municips)

Successfully inserted 80 municipalites.


### 3: Insert into Candidates

In [15]:
candidates = [
    ("DŽAFEROVIĆ ŠEFIK - SDA - STRANKA DEMOKRATSKE AKCIJE", "B"),
    ("BEĆIROVIĆ DENIS - SDP - SOCIJALDEMOKRATSKA PARTIJA BOSNE I HERCEGOVINE", "B"),
    ("RADONČIĆ FAHRUDIN - SBB-FAHRUDIN RADONČIĆ", "B"),
    ("HADŽIKADIĆ MIRSAD - MIRSAD HADŽIKADIĆ-PLATFORMA ZA PROGRES", "B"),
    ("ŠEPIĆ SENAD - NEZAVISNI BLOK", "B"),
    ("JERLAGIĆ AMER - STRANKA ZA BOSNU I HERCEGOVINU", "B"),
    ("KOMŠIĆ ŽELJKO - DEMOKRATSKA FRONTA", "C"),
    ("ČOVIĆ DRAGAN - HDZ BIH-HRVATSKA DEMOKRATSKA ZAJEDNICA BOSNE I HERCEGOVINE", "C"),
    ("ZELENIKA DIANA - HRVATSKA DEMOKRATSKA ZAJEDNICA 1990 - HDZ 1990", "C"),
    ("FALATAR BORIŠA - NAŠA STRANKA", "C"),
    ("IVANKOVIĆ-LIJANOVIĆ JERKO - NARODNA STRANKA RADOM ZA BOLJITAK", "C"),
]
insert_candidates(candidates)

Successfully inserted 11 candidates.


### 4: Insert election results into CandidateResults

In [16]:
year_key = "WebResult_2018GEN_2018_10_4_15_40_5"
modern_scrape(2018, year_key, m_to_c)

Successfully inserted 11 candidate results for velika kladusa.
Successfully inserted 11 candidate results for cazin.
Successfully inserted 11 candidate results for bihac.
Successfully inserted 11 candidate results for bosanska krupa.
Successfully inserted 11 candidate results for buzim.
Successfully inserted 11 candidate results for odzak.
Successfully inserted 11 candidate results for domaljevac samac.
Successfully inserted 11 candidate results for orasje.
Successfully inserted 11 candidate results for gradacac.
Successfully inserted 11 candidate results for brcko distrikt bih.
Successfully inserted 11 candidate results for bosanski petrovac.
Successfully inserted 11 candidate results for sanski most.
Successfully inserted 11 candidate results for doboj istok.
Successfully inserted 11 candidate results for doboj jug.
Successfully inserted 11 candidate results for tesanj.
Successfully inserted 11 candidate results for maglaj.
Successfully inserted 11 candidate results for gracanica.
Su

### 5: Sanity Check

In [17]:
try:
    conn = sqlite3.connect(SAVE_DIR + "political_data.db")

    sql_query = """
        SELECT
            m.name AS municipality_name,
            y.year,
            c.name,
            c.ethnicity,
            cr.votes
        FROM 
            Municipalities as m
        JOIN
            Years AS y ON m.year_id = y.year_id
        JOIN
            CandidateResults AS cr ON m.municipality_id = cr.municipality_id
        JOIN
            Candidates AS c ON cr.candidate_id = c.candidate_id
        WHERE y.year = 2018
    """

    df = pd.read_sql_query(sql_query, conn)
except sqlite3.Error as e:
    print(f"An error occurred: {e}")
finally:
    conn.close()

In [18]:
df

,municipality_name,year,name,ethnicity,votes
0,velika kladusa,2018,DŽAFEROVIĆ ŠEFIK - SDA - STRANKA DEMOKRATSKE A...,B,2691
1,velika kladusa,2018,BEĆIROVIĆ DENIS - SDP - SOCIJALDEMOKRATSKA PAR...,B,2545
2,velika kladusa,2018,KOMŠIĆ ŽELJKO - DEMOKRATSKA FRONTA,C,2342
3,velika kladusa,2018,HADŽIKADIĆ MIRSAD - MIRSAD HADŽIKADIĆ-PLATFORM...,B,1313
4,velika kladusa,2018,RADONČIĆ FAHRUDIN - SBB-FAHRUDIN RADONČIĆ,B,1090
...,...,...,...,...,...
875,grad mostar,2018,HADŽIKADIĆ MIRSAD - MIRSAD HADŽIKADIĆ-PLATFORM...,B,1471
876,grad mostar,2018,FALATAR BORIŠA - NAŠA STRANKA,C,977
877,grad mostar,2018,ŠEPIĆ SENAD - NEZAVISNI BLOK,B,472
878,grad mostar,2018,JERLAGIĆ AMER - STRANKA ZA BOSNU I HERCEGOVINU,B,259


## 2014

In [19]:
insert_year(2014)

Successfully inserted 1 year.


### 2: Insert into Municipalities

`municips` defined in 2022 above in the same step.

In [20]:
insert_municipalities(2014, municips)

Successfully inserted 80 municipalites.


### 3: Insert into Candidates

In [21]:
candidates = [
    ("IZETBEGOVIĆ BAKIR - SDA - STRANKA DEMOKRATSKE AKCIJE", "B"),
    ("RADONČIĆ FAHRUDIN - SBB - FAHRUDIN RADONČIĆ", "B"),
    ("SULJAGIĆ EMIR - DEMOKRATSKA FRONTA-ŽELJKO KOMŠIĆ", "B"),
    ("HADŽIOMEROVIĆ BAKIR - SDP-SOCIJALDEMOKRATSKA PARTIJA BIH", "B"),
    ("HALILOVIĆ SEFER - BPS-SEFER HALILOVIĆ", "B"),
    ("CERIĆ MUSTAFA - CERIĆ MUSTAFA", "B"),
    ("BAJRAMOVIĆ DŽEBRAIL - STRANKA DIJASPORE BOSNE I HERCEGOVINE", "B"),
    ("KEBO MIRSAD - KEBO MIRSAD", "B"),
    ("TUZLIĆ HALIL - TUZLIĆ HALIL", "B"),
    ("ŽIGIĆ ADIL - ŽIGIĆ ADIL", "B"),
    ("ČOVIĆ DRAGAN - HDZ BIH - HRVATSKA DEMOKRATSKA ZAJEDNICA BOSNE I HERCEGOVINE", "C"),
    ("RAGUŽ MARTIN - HDZ 1990 HRVATSKA DEMOKRATSKA ZAJEDNICA", "C"),
    ("BUDIMIR ŽIVKO - STRANKA PRAVDE I POVJERENJA", "C"),
    ("POPOVIĆ ANTO - DEMOKRATSKA FRONTA-ŽELJKO KOMŠIĆ", "C"),
]
insert_candidates(candidates)

Successfully inserted 14 candidates.


### 4: Insert election results into CandidateResults

In [22]:
url = "https://www.izbori.ba/Potvrdjeni2014/Finalni/PredsjednistvoBiH/Nivo701702.aspx"
headless = True
year = 2014
# m_to_c defined in 2022 section

older_scrape(url, headless, year, m_to_c)

Starting browser.
Successfully found 80 links. Starting iteration...
--- Loop 1 of 80 ---
Clicking link: 001 - VELIKA KLADUŠA
Scraped 14 candidate results for 001 - VELIKA KLADUŠA.
Successfully inserted 14 candidate results for velika kladusa.
--- Loop 2 of 80 ---
Clicking link: 002 - CAZIN
Scraped 14 candidate results for 002 - CAZIN.
Successfully inserted 14 candidate results for cazin.
--- Loop 3 of 80 ---
Clicking link: 003 - BIHAĆ
Scraped 14 candidate results for 003 - BIHAĆ.
Successfully inserted 14 candidate results for bihac.
--- Loop 4 of 80 ---
Clicking link: 004 - BOSANSKA KRUPA
Scraped 14 candidate results for 004 - BOSANSKA KRUPA.
Successfully inserted 14 candidate results for bosanska krupa.
--- Loop 5 of 80 ---
Clicking link: 005 - BUŽIM
Scraped 14 candidate results for 005 - BUŽIM.
Successfully inserted 14 candidate results for buzim.
--- Loop 6 of 80 ---
Clicking link: 017 - ODŽAK
Scraped 14 candidate results for 017 - ODŽAK.
Successfully inserted 14 candidate results 

### 5: Sanity Check

In [23]:
try:
    conn = sqlite3.connect(SAVE_DIR + "political_data.db")

    sql_query = """
        SELECT
            m.name AS municipality_name,
            y.year,
            c.name,
            c.ethnicity,
            cr.votes
        FROM 
            Municipalities as m
        JOIN
            Years AS y ON m.year_id = y.year_id
        JOIN
            CandidateResults AS cr ON m.municipality_id = cr.municipality_id
        JOIN
            Candidates AS c ON cr.candidate_id = c.candidate_id
        WHERE y.year = 2014
    """

    df = pd.read_sql_query(sql_query, conn)
except sqlite3.Error as e:
    print(f"An error occurred: {e}")
finally:
    conn.close()

In [24]:
df

,municipality_name,year,name,ethnicity,votes
0,velika kladusa,2014,RADONČIĆ FAHRUDIN - SBB - FAHRUDIN RADONČIĆ,B,3210
1,velika kladusa,2014,IZETBEGOVIĆ BAKIR - SDA - STRANKA DEMOKRATSKE ...,B,3034
2,velika kladusa,2014,BUDIMIR ŽIVKO - STRANKA PRAVDE I POVJERENJA,C,1400
3,velika kladusa,2014,HADŽIOMEROVIĆ BAKIR - SDP-SOCIJALDEMOKRATSKA P...,B,1390
4,velika kladusa,2014,SULJAGIĆ EMIR - DEMOKRATSKA FRONTA-ŽELJKO KOMŠIĆ,B,917
...,...,...,...,...,...
1115,grad mostar,2014,POPOVIĆ ANTO - DEMOKRATSKA FRONTA-ŽELJKO KOMŠIĆ,C,280
1116,grad mostar,2014,KEBO MIRSAD - KEBO MIRSAD,B,171
1117,grad mostar,2014,BAJRAMOVIĆ DŽEBRAIL - STRANKA DIJASPORE BOSNE ...,B,82
1118,grad mostar,2014,TUZLIĆ HALIL - TUZLIĆ HALIL,B,58


It works

## 2010

In [25]:
insert_year(2010)

Successfully inserted 1 year.


### 2: Insert into Municipalities

`municips` defined in 2022 above in the same step.

In [26]:
insert_municipalities(2010, municips)

Successfully inserted 80 municipalites.


### 3: Insert into Candidates

In [27]:
candidates = [
    ("IZETBEGOVIĆ BAKIR - SDA - STRANKA DEMOKRATSKE AKCIJE", "B"),
    ("RADONČIĆ FAHRUDIN - SAVEZ ZA BOLJU BUDUĆNOST BIH - SBB BIH FAHRUDIN RADONČIĆ", "B"),
    ("SILAJDŽIĆ HARIS - STRANKA ZA BOSNU I HERCEGOVINU", "B"),
    ("ĐEDOVIĆ IBRAHIM - DEMOKRATSKA NARODNA ZAJEDNICA - DNZ BIH", "B"),
    ("DEMIROVIĆ MUJO - BPS - SEFER HALILOVIĆ", "B"),
    ("LATIĆ ĐEMAL - STRANKA DEMOKRATSKE AKTIVNOSTI A-sda", "B"),
    ("SPAHIĆ IBRAHIM - GRAĐANSKA DEMOKRATSKA STRANKA BOSNE I HERCEGOVINE", "B"),
    ("KEŠETOVIĆ IZUDIN - BOSS - BOSANSKA STRANKA - MIRNES AJANOVIĆ", "B"),
    ("JUSIĆ AIDA - JUSIĆ AIDA - NEZAVISNI KANDIDAT", "B"),
    ("KOMŠIĆ ŽELJKO - SDP - SOCIJALDEMOKRATSKA PARTIJA BIH", "C"),
    ("KRIŠTO BORJANA - HDZ BIH - HRVATSKA DEMOKRATSKA ZAJEDNICA BOSNE I HERCEGOVINE", "C"),
    ("RAGUŽ MARTIN - HRVATSKA KOALICIJA HDZ 1990 - HSP BIH", "C"),
    ("IVANKOVIĆ-LIJANOVIĆ JERKO - NARODNA STRANKA RADOM ZA BOLJITAK", "C"),
    ("GALIĆ PERO - GALIĆ PERO - NEOVISNI KANDIDAT", "C"),
    ("KUTLE MILE - KUTLE MILE - NEOVISNI KANDIDAT", "C"),
    ("GALIĆ FERDO - GALIĆ FERDO - NEOVISNI KANDIDAT", "C"),
]
insert_candidates(candidates)

Successfully inserted 14 candidates.


### 4: Insert election results into CandidateResults

# TODO

Even this website is not matched to 2014 ;(. So I gotta find a way to unify it...

In [ ]:
url = "https://www.izbori.ba/Finalni2010/Finalni/PredsjednistvoBiH/Nivo701702.aspx"
headless = True
year = 2010
# m_to_c defined in 2022 section



### 5: Sanity Check

In [29]:
try:
    conn = sqlite3.connect(SAVE_DIR + "political_data.db")

    sql_query = """
        SELECT
            m.name AS municipality_name,
            y.year,
            c.name,
            c.ethnicity,
            cr.votes
        FROM 
            Municipalities as m
        JOIN
            Years AS y ON m.year_id = y.year_id
        JOIN
            CandidateResults AS cr ON m.municipality_id = cr.municipality_id
        JOIN
            Candidates AS c ON cr.candidate_id = c.candidate_id
        WHERE y.year = 2010
    """

    df = pd.read_sql_query(sql_query, conn)
except sqlite3.Error as e:
    print(f"An error occurred: {e}")
finally:
    conn.close()

In [ ]:
df

It works